### Scoring Algorithm

In [4]:
import pandas as pd

try:
    #Import applicant data
    df = pd.read_csv('applicants.csv')

    #Number of applicants to select (ITERATION 2: MAKE DYNAMIC)
    N = 5

    #Clean column names
    df.columns = df.columns.str.strip()

    #Copy for scoring
    df_score = df.copy()

    #Transform binary columns for scoring
    df_score['Past Volunteer (score)'] = df_score['Past Volunteer'].map({'Yes': 2, 'No': 0})
    df_score['Past Camper (score)'] = df_score['Past Camper'].map({'Yes': 1, 'No': 0})

    #Add count columns (keep raw columns intact)
    df_score['Course Count'] = df_score['Courses'].apply(lambda x: len(str(x).split(';')))
    df_score['Technical Skill Count'] = df_score['Technical Skills'].apply(lambda x: len(str(x).split(';')))

    #Map experience columns
    experience_map = {'0 hrs': 0, '1-9 hrs': 1, '10-20 hrs': 2, '20+ hrs': 3}
    df_score['Youth Experience (score)'] = df_score['Youth Experience'].map(experience_map)
    df_score['Leadership Experience (score)'] = df_score['Leadership Experience'].map(experience_map)

    #Calculate score based on counts and scoring columns
    df_score['Score'] = (
        df_score['Past Volunteer (score)'] +
        df_score['Past Camper (score)'] +
        df_score['Course Count'] +
        df_score['Technical Skill Count'] +
        df_score['Youth Experience (score)'] +
        df_score['Leadership Experience (score)']
    )

    #Sort by score and assign rank
    df_score = df_score.sort_values(by='Score', ascending=False).reset_index(drop=True)
    df_score.insert(0, 'Rank', df_score.index + 1)

    #Top N scorers
    df_score_top = df_score.head(N)

    #Save outputs
    df_score_top.to_csv("top_" + str(N) + "_applicants.csv", index=False)
except:
    print('Something went wrong. Please check the input file and try again.')


### Scheduling Optimization

In [5]:
#Import relevant data (Qualtrics formatted export)
availability_df = pd.read_csv('Capstone Sample Weekly Scheduling Data.csv', skiprows=[0,1])

#Clean column names and format
new_header = ['Name', 'Email', 'Week 1', 'Week 2', 'Week 3', 'Week 4', 'Week 5', 'Week 6', 'Week 7']
availability_df.columns = new_header
availability_df.insert(0, 'Volunteer ID', availability_df.index + 1)
availability_map = {'Yes': 1, 'No': 0}

for col in availability_df.columns[3:]:
    availability_df[col] = availability_df[col].map(availability_map)

#Sample weekly volunteer requirements N_j (ITERATION 2: MAKE DYNAMIC)
weekly_volunteers = [3,2,4,5,4,3,2]

#availability_df

In [ ]:
import pulp

# Create model
model = pulp.LpProblem("Scheduling_Optimization", pulp.LpMaximize)

# Set volunteers and weeks
volunteers = availability_df['Volunteer ID'].tolist()
weeks = list(range(1, 8))

# Decision variables (binary assignment)
week_match = pulp.LpVariable.dicts(
    "x",
    [(i, j) for i in volunteers for j in weeks],
    cat='Binary'
)

# Count available weeks per volunteer
availability_counts = {
    i: sum(
        availability_df.loc[
            availability_df['Volunteer ID'] == i,
            availability_df.columns[3:]  # week columns
        ].values[0]
    )
    for i in volunteers
}

# Create priority weights
max_availability = max(availability_counts.values())
weights = {
    i: (max_availability - availability_counts[i] + 1)
    for i in volunteers
}

# -------------------
# Constraints
# -------------------

# Each volunteer must be assigned at least once (if available)
for i in volunteers:
    model += (
        pulp.lpSum(
            week_match[(i, j)] * availability_df.iloc[i-1, j+2]
            for j in weeks
        ) >= 1,
        f"Volunteer_{i}_Min_Placement"
    )

# Each week must have required number of volunteers
for j in weeks:
    model += (
        pulp.lpSum(
            week_match[(i, j)] * availability_df.iloc[i-1, j+2]
            for i in volunteers
        ) == weekly_volunteers[j-1],
        f"Week_{j}_Min_Volunteers"
    )

# -------------------
# Objective Function
# -------------------

model += pulp.lpSum(
    weights[i] * week_match[(i, j)] * availability_df.iloc[i-1, j+2]
    for i in volunteers
    for j in weeks
)

# Solve model
model.solve()

# -------------------
# Results
# -------------------
print("Status:", pulp.LpStatus[model.status])
print("Objective value:", pulp.value(model.objective))

print("\nAssigned Volunteers per Week:")
for i in volunteers:
    for j in weeks:
        if pulp.value(week_match[(i, j)] * availability_df.iloc[i-1, j+2]) > 0.5:
            print(f" - Volunteer {i} assigned to Week {j}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/taylorliew/anaconda3/envs/MSCI446/lib/python3.11/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/0q/wgx2cs755472fdwvfvs4d2c00000gn/T/d3a1c60f66564e2b9b8fb6ef9cebfd11-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/0q/wgx2cs755472fdwvfvs4d2c00000gn/T/d3a1c60f66564e2b9b8fb6ef9cebfd11-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 22 COLUMNS
At line 193 RHS
At line 211 BOUNDS
At line 246 ENDATA
Problem MODEL has 17 rows, 34 columns and 68 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 64 - 0.00 seconds
Cgl0004I processed model has 11 rows, 27 columns (27 integer (25 of which binary)) and 45 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -64

### Export Results

In [9]:
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill

#Create workbook and sheet
wb = Workbook()
ws = wb.active
ws.title = "Schedule"

#Cols for each week
cell_mapping = [('A', 'B'), ('C', 'D'), ('E', 'F'), ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'N')]

#Style
header_font = Font(bold=True)
header_fill = PatternFill(start_color="daa2fc", end_color="daa2fc", fill_type="solid")

#Fill weekly lists with names and emails
week_number = 1
for start_cell, end_cell in cell_mapping:
    ws.merge_cells(start_cell + str(1) + ':' + end_cell + str(1))
    cell = ws[start_cell + str(1)]
    cell.value = f"Week {week_number}"
    cell.font = header_font
    cell.fill = header_fill
    row = 2

    for i in volunteers:
        if pulp.value(week_match[i, week_number] * availability_df.iloc[i-1, week_number+2]) > 0.5:
            ws[f"{start_cell}{row}"] = availability_df.loc[availability_df['Volunteer ID'] == i, 'Name'].values[0]
            ws[f"{end_cell}{row}"] = availability_df.loc[availability_df['Volunteer ID'] == i, 'Email'].values[0]

            row += 1
    week_number += 1

#Save file
wb.save("Volunteer_Schedule.xlsx")
